In [ ]:
#| default_exp installing

In [ ]:
#| export
import shutil
import json
import docker
import os

In [ ]:
#| export

def upload_image_to_dockerhub(image_tag:str, dockerhub_user = None, dockerhub_pass = None):
    "Uploads a docker image to dockerhub. If username or password is not given, environmental variables DOCKERHUB_USER and DOCKERHUB_PASSWORD will be used instead"
    if not dockerhub_user or not dockerhub_pass:
        dockerhub_user = os.getenv("DOCKERHUB_USER")
        dockerhub_pass = os.getenv("DOCKERHUB_PASSWORD")
        if not dockerhub_user:
            raise Exception("DOCKERHUB_USER environment variable not defined")
        if not dockerhub_pass:
            raise Exception("DOCKERHUB_PASSWORD environment variable not defined")

    dockerhub_tag = dockerhub_user+'/'+image_tag
    client = docker.from_env()
    client.login(dockerhub_user, dockerhub_pass)
    if not docker.APIClient().tag(image_tag, dockerhub_tag):
        raise Exception(f"Failed to tag image: {image_tag} => {dockerhub_tag}")
    client.api.push(dockerhub_tag)

In [ ]:
#| export
    
class ArduinoNano33BLE_Installer:
    def __init__(self):
        self.ARDUINO_DIR = "arduino"
        self.CODE_DIR = self.ARDUINO_DIR + "/template"
        self.DOCKER_IMG = "nano33ble"
    
    def __convert_model_to_arduino(self, model_path : str):
        "Copies model.cc file to arduino directory and appends necessary include and const to the start of the file."
        with open("arduino/template/person_detect_model_data.cpp", "w") as model_file:
            model_file.write("#include \"person_detect_model_data.h\"\n")
            with open(f"{model_path}/model.cc") as model_data:
                model_file.write(model_data.read())
            
    def compile(self, model_path : str):
        "Compiles arduino sketch in docker, using model file at `model_path` in local filesystem"
        self.__convert_model_to_arduino(model_path)
        client = docker.from_env()
        image = client.images.build(path=self.ARDUINO_DIR, tag=self.DOCKER_IMG)
        

    def upload(self, dockerhub_user=None, dockerhub_pass=None):
        "Uploads compiled sketch inside docker image to dockerhub"
        upload_image_to_dockerhub(self.DOCKER_IMG, dockerhub_user, dockerhub_pass)
        

    def list_ports(self):
        "Returns a list of ports of connected devices"
        ports = []
        cmd_output = subprocess.run(['docker run --privileged nano33ble board list --format=json'], shell=True,capture_output=True).stdout
        result = json.loads(cmd_output.decode("ascii"))
        for row in result:
            ports.append(row["port"]["address"])
        return ports

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
show_doc(ArduinoNano33BLE_Installer.compile)

---

### ArduinoNano33BLE_Installer.compile

>      ArduinoNano33BLE_Installer.compile (model_path:str)

Compiles arduino sketch in docker, using model file at `model_path` in local filesystem

In [ ]:
show_doc(ArduinoNano33BLE_Installer.compile)

NameError: name 'show_doc' is not defined